# Train Graph NN on Call Mentions

In [1]:
# Settings
model_name = 'exclude_previous_rating_model_2'
inductive_or_transductive = 'Inductive'

In [2]:
# Packages
from Graph_NN_Functions import *

## Load Learnable Network Data

In [3]:
# Load feature and class data
feature_and_class_df = pd.read_excel('../../../Data/Learnable Network/feature_and_class_df.xlsx')
feature_and_class_df

,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,...,Ovrst,Undrst,PN,SW,AP,OU,tone,num_q_by_len,train_test_80_20,node
0,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,AA,2015-02-18,AAA,2014-04-24,2015-02-18,...,364,131,5.518519,15.261905,2.661290,2.778626,3.188264,0.003822,train,272
1,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,AA,2015-02-18,AAA,2014-04-24,2015-02-18,...,465,152,5.348485,15.934783,3.296482,3.059211,3.681858,0.002766,train,1499
2,AAPL,2015-04-01,2015-01-27,AA,2015-02-18,AA,2015-05-28,AA,2014-05-27,2015-05-28,...,468,151,3.927711,8.113636,2.841346,3.099338,1.307366,0.004628,train,814
3,AAPL,2015-07-01,2015-04-27,AA,2015-06-02,AA,2015-08-25,AA,2015-05-28,2015-08-25,...,415,135,5.250000,9.142857,2.640187,3.074074,2.025933,0.003861,train,1888
4,AAPL,2015-10-01,2015-07-21,AA,2015-08-25,AA,2016-05-20,AA,2015-06-02,2016-05-20,...,449,148,4.209877,10.442857,2.579909,3.033784,1.815531,0.003915,train,1224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,ZBRA,2016-01-01,2015-11-10,B,2015-09-03,B,2016-09-26,NaN,NaN,2016-09-26,...,431,202,2.662338,13.586667,2.768293,2.133663,0.982796,0.005321,train,369
1960,ZBRA,2016-04-01,2016-02-25,B,2015-09-03,B,2016-09-26,NaN,NaN,2016-09-26,...,449,173,3.991071,13.574713,2.382550,2.595376,1.925784,0.002704,train,839
1961,ZBRA,2016-07-01,2016-05-10,B,2015-09-03,B,2016-09-26,NaN,NaN,2016-09-26,...,394,197,2.671053,8.080357,2.446970,2.000000,-0.423271,0.002775,train,448
1962,ZBRA,2016-10-01,2016-08-09,B,2016-09-26,NaN,NaN,B,2015-09-03,2016-12-31,...,403,183,3.625954,12.151163,3.066116,2.202186,1.210391,0.003215,test,1286


In [4]:
src_dst_df = pd.read_excel('../../../Data/Learnable Network/src_dst_df.xlsx')
src_dst_df

,src,dst
0,1487,1611
1,1010,1611
2,127,1611
3,858,1611
4,333,1611
...,...,...
1801,1814,1851
1802,1556,1859
1803,908,813
1804,123,610


In [5]:
node_to_int = pd.read_excel('../../../Data/Learnable Network/node_to_int.xlsx')
# convert to dictionary
node_to_int = node_to_int.set_index('node').to_dict()['node_int']
node_to_int

{'SWN : 2015-04-01': 0,
 'DTE : 2013-10-01': 1,
 'CSGS : 2015-07-01': 2,
 'MSI : 2016-07-01': 3,
 'TWI : 2015-10-01': 4,
 'IRM : 2015-07-01': 5,
 'NVDA : 2014-04-01': 6,
 'AXL : 2015-10-01': 7,
 'MSI : 2014-01-01': 8,
 'TUP : 2016-04-01': 9,
 'MKC : 2015-07-01': 10,
 'SLCA : 2016-01-01': 11,
 'KMI : 2015-04-01': 12,
 'DVN : 2015-01-01': 13,
 'MTZ : 2016-04-01': 14,
 'IAC : 2016-07-01': 15,
 'IBM : 2015-07-01': 16,
 'CRK : 2015-01-01': 17,
 'DHR : 2016-07-01': 18,
 'POR : 2013-01-01': 19,
 'DAN : 2013-07-01': 20,
 'TDS : 2013-07-01': 21,
 'SBUX : 2016-04-01': 22,
 'ABBV : 2015-04-01': 23,
 'NSC : 2014-04-01': 24,
 'XEL : 2016-07-01': 25,
 'FLR : 2015-01-01': 26,
 'AVT : 2015-07-01': 27,
 'COP : 2015-10-01': 28,
 'MPC : 2014-10-01': 29,
 'RRC : 2015-10-01': 30,
 'NRG : 2016-01-01': 31,
 'MUR : 2014-04-01': 32,
 'SBUX : 2013-04-01': 33,
 'UIS : 2015-01-01': 34,
 'KSS : 2011-10-01': 35,
 'OMC : 2016-04-01': 36,
 'SBUX : 2011-10-01': 37,
 'TCS : 2015-10-01': 38,
 'DVN : 2012-10-01': 39,
 'P

In [6]:
# Load column names
numeric_feature_columns, cat_feature_columns, target_column, custom_mapping = get_column_names_and_mapping(model_name)

In [7]:
# Prepare matrices
X_train_scaled, X_test_scaled, y_train, y_test, feature_names, train_ticker_by_fixed_quarter_date, test_ticker_by_fixed_quarter_date = prepare_matrices(feature_and_class_df, numeric_feature_columns, cat_feature_columns, target_column, custom_mapping)

feature names: 
['num__EBIT' 'num__common_plus_preferred_stock' 'num__workingCapital'
 'num__Ratio_A' 'num__Ratio_B' 'num__Ratio_C' 'num__Ratio_D'
 'num__Ratio_E' 'num__cashAndCashEquivalents' 'num__shortTermInvestments'
 'num__cashAndShortTermInvestments' 'num__netReceivables'
 'num__inventory_balance_sheet' 'num__otherCurrentAssets'
 'num__totalCurrentAssets' 'num__propertyPlantEquipmentNet'
 'num__goodwill' 'num__intangibleAssets'
 'num__goodwillAndIntangibleAssets' 'num__longTermInvestments'
 'num__taxAssets' 'num__otherNonCurrentAssets'
 'num__totalNonCurrentAssets' 'num__otherAssets' 'num__totalAssets'
 'num__accountPayables' 'num__shortTermDebt' 'num__taxPayables'
 'num__deferredRevenue' 'num__otherCurrentLiabilities'
 'num__totalCurrentLiabilities' 'num__longTermDebt'
 'num__deferredRevenueNonCurrent' 'num__deferredTaxLiabilitiesNonCurrent'
 'num__otherNonCurrentLiabilities' 'num__totalNonCurrentLiabilities'
 'num__otherLiabilities' 'num__capitalLeaseObligations'
 'num__totalLi

In [8]:
# Assemble back into dataframes

# Train and val
train_and_val_df = pd.DataFrame(X_train_scaled, columns=feature_names)
print(train_and_val_df)
# Add y_train
train_and_val_df[target_column] = y_train.reset_index(drop=True)
# Add ticker by fixed quarter date
train_and_val_df = pd.concat([train_ticker_by_fixed_quarter_date.reset_index(drop=True).sort_values(['ticker', 'fixed_quarter_date']), train_and_val_df], axis=1)
# Add node by merging with feature_and_class_df (inner join)
train_and_val_df = train_and_val_df.merge(feature_and_class_df[['ticker', 'fixed_quarter_date', 'node']], on=['ticker', 'fixed_quarter_date'], how='inner')
# Drop ticker and fixed_quarter_date
train_and_val_df = train_and_val_df.drop(['ticker', 'fixed_quarter_date'], axis=1)

# Test
test_df = pd.DataFrame(X_test_scaled, columns=feature_names)
# Add y_test
test_df[target_column] = y_test.reset_index(drop=True)
# Add ticker by fixed quarter date
test_df = pd.concat([test_ticker_by_fixed_quarter_date.reset_index(drop=True).sort_values(['ticker', 'fixed_quarter_date']), test_df], axis=1)
# Add node by merging with feature_and_class_df (inner join)
test_df = test_df.merge(feature_and_class_df[['ticker', 'fixed_quarter_date', 'node']], on=['ticker', 'fixed_quarter_date'], how='inner')
# Drop ticker and fixed_quarter_date
test_df = test_df.drop(['ticker', 'fixed_quarter_date'], axis=1)

print('finalized dfs')
print(train_and_val_df)
print(test_df)
print('missing values of target_column in train_and_val_df or test_df?')
print(train_and_val_df[target_column].isnull().sum() > 0)
print(test_df[target_column].isnull().sum() > 0)

      num__EBIT  num__common_plus_preferred_stock  num__workingCapital  \
0     -0.550946                         -0.405512            -0.220795   
1     -0.549646                         -0.404515            -0.224155   
2     -0.530384                         -0.403773            -0.223205   
3     -0.539172                         -0.402764            -0.223332   
4     -0.545349                         -0.401957            -0.224037   
...         ...                               ...                  ...   
1557   0.081791                          1.214090            -0.507520   
1558  -0.466756                         -0.423693            -0.106127   
1559  -0.454378                         -0.423693            -0.133219   
1560  -0.502770                         -0.423457            -0.142898   
1561  -0.271596                         -0.420061             0.254596   

      num__Ratio_A  num__Ratio_B  num__Ratio_C  num__Ratio_D  num__Ratio_E  \
0         1.365738     -0.223037 

## Run the Model

In [9]:
run_model(train_and_val_df = train_and_val_df,
         test_df = test_df,
         src_dst_df = src_dst_df,
         model_dir = '../../../Output/Modelling/Graph Neural Network/' + inductive_or_transductive + '/' + model_name + '/',
         prediction_file_path = '../../../Data/Predictions/Graph Neural Network/' + inductive_or_transductive + '/' + model_name + '_predictions.xlsx',
         target_column = target_column,
         custom_mapping = custom_mapping,
         node_to_int = node_to_int,
         n_hidden = 32,
         n_layers = 2,
         dropout = 0.0,
         weight_decay = 5e-4,
         n_epochs = 100,
         lr = 0.01,
         aggregator_type = "pool",
         inductive = (inductive_or_transductive == 'Inductive'))

Further slice the train dataset into train and validation datasets.
The training data has shape: (1249, 141).
The validation data has shape: (313, 141).
The test data has shape: (402, 141).
Generate train, validation, and test masks.
sum of train mask
tensor(1249)
sum of val mask
tensor(313)
sum of test mask
tensor(402)
Number of nodes = 1964
Number of features for each node = 139
Number of classes = 9.
Initializing Model
Initialized Model
NodeClassification(
  (gconv_model): GraphSAGEModel(
    (layers): ModuleList(
      (0): SAGEConv(
        (feat_drop): Dropout(p=0.0, inplace=False)
        (fc_pool): Linear(in_features=139, out_features=139, bias=True)
        (fc_neigh): Linear(in_features=139, out_features=32, bias=False)
        (fc_self): Linear(in_features=139, out_features=32, bias=True)
      )
      (1): SAGEConv(
        (feat_drop): Dropout(p=0.0, inplace=False)
        (fc_pool): Linear(in_features=32, out_features=32, bias=True)
        (fc_neigh): Linear(in_features=